In [1]:
# ЧЕРНОВОЙ ВАРИАНТ ПО НЕЙРО-КОПИРАЙТЕРУ ДИАЛОГОВ С ПРОМПТАМИ ИЗ СКРИПТОВ ОПЕРАТОРОВ В COLAB

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!rm -rf "/content/drive/MyDrive/KIA_DIALOG/Copywriter/KIA/BASE/"
!mkdir -p "/content/drive/MyDrive/KIA_DIALOG/Copywriter/KIA/BASE/"

In [3]:

KEY="ONLY_6"
ALL_FIRST_TEXTS={
 "ONLY_6": "https://docs.google.com/spreadsheets/d/1VRk8vvIx-u4OjSKx15VNRVQhvw3qO1ug/export?format=xlsx"
, "ONLY_7": "https://docs.google.com/spreadsheets/d/1zLaDn598Mm9zs0w1To55rcpAoeMjfuRH/export?format=xlsx"
}
FIRST_TEXTS={KEY: ALL_FIRST_TEXTS[KEY]}

ALL_FIRST_SHEETS={
  "ONLY_6": "Service Maintenance"
, "ONLY_7": "Systems and Applications"
}
FIRST_SHEETS={KEY: ALL_FIRST_SHEETS[KEY]}

# Установка

In [4]:
# @title Установка пакетов
from IPython.display import clear_output

!pip install --upgrade tiktoken
!pip install langchain openai
!pip install faiss-cpu
!pip install urllib

clear_output()

import getpass
import openai
import os
def get_key_ОpenAI():
  openai.api_key = getpass.getpass(prompt='Введите секретный ключ для сервиса chatGPT: ')
  os.environ["OPENAI_API_KEY"] = openai.api_key

get_key_ОpenAI()


Введите секретный ключ для сервиса chatGPT: ··········


In [5]:
#@title Установка библиотек
!pip install tiktoken langchain openai chromadb gspread oauth2client nltk pydantic==1.10.8 faiss-cpu python-docx

In [6]:
#@title Импорт библиотек и Сервисные функции
import gdown
from IPython.display import clear_output
from langchain.llms import OpenAI
from langchain.docstore.document import Document
import requests
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import CharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
import pathlib
import subprocess
import tempfile

import time
import os
import openai
import tiktoken
import re
import os
import json
from tqdm.auto import tqdm
import getpass


# ----------------------------------
MODEL_TURBO_16K = "gpt-3.5-turbo-16k"
MODEL_TURBO_0613 = "gpt-3.5-turbo-0613"
MODEL_GPT4 = "gpt-4-0613"
# ----------------------------------

clear_output()

class WorkerОpenAI():
  def __init__(self, \
               system_promt = " ", \
               system_promt_lector = " ", \
               mod = MODEL_TURBO_16K, \
               content_topics = None, \
               save_project = '/content/'):
    self.model = mod
    self.save_project  = save_project

    if content_topics:
      self.content_topics = self.load_txt_file(content_topics)

    # системные настройки
    self.system_promt = self.load_document_text(system_promt)
    self.speaker_system_promt = self.load_document_text(system_promt_lector)


  def load_document_text(self, url: str) -> str:
      # функция для загрузки документа по ссылке из гугл док
      match_ = re.search('/document/d/([a-zA-Z0-9-_]+)', url)
      if match_ is None:
          raise ValueError('Invalid Google Docs URL')
      doc_id = match_.group(1)
      response = requests.get(f'https://docs.google.com/document/d/{doc_id}/export?format=txt')
      response.raise_for_status()
      text = response.text
      return text


  def load_txt_file(self, file_path):
      with open(file_path, 'r') as file_:
          text = file_.read()
      return text

  # пример подсчета токенов
  def num_tokens_from_messages(self, messages):
      """Returns the number of tokens used by a list of messages."""
      try:
          encoding = tiktoken.encoding_for_model(self.model)
      except KeyError:
          encoding = tiktoken.get_encoding("cl100k_base")
      # if self.model == "gpt-3.5-turbo-0301":  # note: future models may deviate from this
      if self.model in ["gpt-3.5-turbo-0613", "gpt-3.5-turbo-16k", "gpt-4-0613"]:  # note: future models may deviate from this
          num_tokens = 0
          for message in messages:
              num_tokens += 4  # every message follows <im_start>{role/name}\n{content}<im_end>\n
              for key, value in message.items():
                  num_tokens += len(encoding.encode(value))
                  if key == "name":  # if there's a name, the role is omitted
                      num_tokens += -1  # role is always required and always 1 token
          num_tokens += 2  # every reply is primed with <im_start>assistant
          return num_tokens
      else:
          raise NotImplementedError(f"""num_tokens_from_messages() is not presently implemented for model {self.model}.
  See https://github.com/openai/openai-python/blob/main/chatml.md for information on how messages are converted to tokens.""")


  def f00_create_embedding_faiss_db(self, doc_txt_dir="/content/", \
                                faiss_db_dir ="/content/", \
                                start_idx = 0, \
                                collection_name = " "):

    def f01_num_tokens_from_string(string: str, encoding_name: str) -> int:
      """Returns the number of tokens in a text string."""
      encoding = tiktoken.get_encoding(encoding_name)
      num_tokens = len(encoding.encode(string))
      return num_tokens

    # Для Копирайтера
    self.splitter = RecursiveCharacterTextSplitter(['\n\n', '\n', ' '], chunk_size=1024, chunk_overlap=300)
    idx_file_folder = start_idx-1
    chunkID = idx_file_folder

    count_tokens = 0
    # проходимся по всем данным
    for _, file_ in enumerate(sorted(os.listdir(doc_txt_dir))):
        print("Загружается файл: ", file_)
        self.file_name = file_
        idx_file_folder +=1
        source_chunks = []
        # разбиваем на несколько частей с помощью метода split_text
        with open(doc_txt_dir + file_, "r") as f:
          for chunk in self.splitter.split_text(f.read()):
              chunkID += 1
              source_chunks.append(Document(page_content=chunk, \
                                  metadata={'source': file_,
                                            'chunkID': chunkID,
                                            "collection_name": collection_name,
                                            'idx_file_folder': idx_file_folder}))


        # Создание индексов документа и СОХРАНЕНИЕ
        # Если документ не пуст, то создать и сохранить базу индексов эмбеддингов отрезков документа
        if len(source_chunks) > 0:
            self.db = FAISS.from_documents(source_chunks, OpenAIEmbeddings())
            count_token = f01_num_tokens_from_string(' '.join([x.page_content for x in source_chunks]), "cl100k_base")
            count_tokens += count_token
            print('Количество токенов в документе :', count_token)
            # print('ЦЕНА запроса:', 0.0004 * (count_token / 1000), ' $')

            self.db.save_local(os.path.join(faiss_db_dir, collection_name, f"{str(idx_file_folder)}_db_initial__{file_[:20]}"))

    print('\nЦЕНА запроса создания базы индексов:', 0.0004 * (count_tokens / 1000), ' $')

  # ЗАПРС в ChatGPT
  def get_ChatCompletion(self, model,  # указываем модель
                         messages,     # словарь запроса
                         temp=0.1):    # температуру

      completion = openai.ChatCompletion.create(
        model= model,
        messages= messages,
        temperature= temp
        )

      print(f'{completion["usage"]["total_tokens"]} токенов использовано всего (вопрос-ответ).')
      # print('ЦЕНА запроса с ответом :', 0.0015*(completion["usage"]["total_tokens"]/1000), ' $')
      print('===========================================: \n')
      return completion.choices[0].message.content

  # ЗАПРС на создание темы и подтемы по Материалам
  def get_search_materials_topics_subtopics(self, materials, model_topics = "gpt-3.5-turbo-16k"):

    messages = [
        {"role": "system", "content": f"{self.system_promt}"},
        {"role": "user", "content": f"""Проанализируй отрывок материала из диалога с другими клиентами: {materials}.
• дай компактный, сжатый, обобщенный список тем и подтем.
• Темы Необходимо оформить #, а подтемы оформить ##_.
• Используй только такой пример, ничего не добавляй лишнего.
• Пример составления списка:
Пример составления списка:
_#...
##_...
_#...
##_...
        """}
        ]

    # example token count from the function defined above
    print(f"{self.num_tokens_from_messages(messages=messages)} токенов использовано на вопрос \n")
    try:
      self.content_topics += self.get_ChatCompletion(model_topics, messages)
    except:
      print("Модель в настоящее время перегружена. Попробуйте позже.")


  # ПОИСК ТЕМЫ и ПОДТЕМЫ
  def search_topics_subtopics(self, num_chunk = 10):
    self.content_topics = ''
    materials = ""
    # Выбираем блоки ПОДРЯД
    len_chunk = len(self.db.docstore._dict)
    if len_chunk < num_chunk + 1:
        for _, doc in self.db.docstore._dict.items():
            materials += f"{doc.page_content}\n"
        self.get_search_materials_topics_subtopics(materials)
    else:
        for ind, (key, doc) in enumerate(self.db.docstore._dict.items()):
          materials += f"{doc.page_content}\n"
          if (ind+1) % num_chunk == 0:
            self.get_search_materials_topics_subtopics(materials)
            materials = ""
    if materials != "":
        self.get_search_materials_topics_subtopics(materials)

    print('Собрали список тем и подтем: ')
    print(self.content_topics)

    with open(f'{self.save_project}{self.file_name[:20]}__темы_подтемы.txt', "w") as f:
      f.write(self.content_topics)


  # ОБЪЕДИНЯЕМ схожие ТЕМЫ
  def get_merge_topics(self, model_topics = "gpt-3.5-turbo-16k"):

    messages = [
        {"role": "system", "content": f"{self.system_promt}"},
        {"role": "user", "content": f"""Проанализируй Темы и подтемы из Текста диалогов специалиста с другими клиентами: {self.content_topics}.

Необходимо объединить похожие по смыслу темы или подтемы, записать компактно.
При необходимости перефразировать тему или подтему. Дай корректный список.
Темы оформи _#, а подтемы оформи ##_.
Используй только такой пример, ничего не добавляй лишнего.
Пример составления списка:
_#...
##_...
_#...
##_...
"""}
]
    # example token count from the function defined above
    print(f"{self.num_tokens_from_messages(messages=messages)} токенов использовано на вопрос \n")
    try:
      self.content_topics = self.get_ChatCompletion(model_topics, messages)
      print('Итоговый список тем и подтем: ')
      print(self.content_topics)
      with open(f'{self.save_project}{self.file_name[:20]}__темы_подтемы_ИТОГ.txt', "w") as f:
        f.write(self.content_topics)
    except:
      print("Модель в настоящее время перегружена. Попробуйте позже.")

  # ОРГАНИЗУЕМ текст блоками
  def organize_text(self, topic, num_chunks, model_topics):

    # Выборка документов по схожести с подтемой
    docs = self.db.similarity_search(topic, k = num_chunks)
    message_content = '\n'.join([doc.page_content + '\n' for i, doc in enumerate(docs)])

    messages = [
        {"role": "system", "content": f"{self.speaker_system_promt}"},
        {"role": "user", "content": f"""Вот отрывки Вашей лекции:\n{message_content}

Опираясь только на информацию с Лекции, указанной выше, расскажите подробнее по Теме: {topic}.
"""}
      ]

    # example token count from the function defined above
    print('\n ===========================================: ')
    print(f"{self.num_tokens_from_messages(messages=messages)} токенов использовано на вопрос \n")
    try:
      self.final_text += self.get_ChatCompletion(model_topics, messages)
    except:
      print("Модель в настоящее время перегружена. Попробуйте позже.")

  # ОРГАНИЗУЕМ ИТОГОВЫЙ ТЕКСТ
  def organize_final_text(self, num_chunks = 10,
                          model_topics = "gpt-3.5-turbo-16k",
                          name = None,
                          db_path = None):
    if name:
      self.file_name = name

    if db_path:
      for curr_base in os.listdir(db_path):
          self.db = FAISS.load_local(os.path.join(db_path, curr_base), OpenAIEmbeddings())

    self.final_text = ""
    list_topics = self.content_topics.split('\n')
    # проходимся по списку тем и подтем
    for ind, topic in tqdm(enumerate(list_topics)):
        # тему просто записываем в итоговый текст
        if ("_#" in topic) and not("_#" in list_topics[ind+1]):
          self.final_text += f'\n{topic}\n'
        # подтему передаем в ChatGPT
        else:
          self.final_text += f'\n{topic}\n'
          self.organize_text(topic, num_chunks, model_topics)

    with open(f'{self.save_project}{self.file_name[:20]}__final_text.txt', "w") as f:
      f.write(self.final_text)
    print("\nСоздали финальный документ для Базы Знаний (final_text)")





# Copywriter. Нейро-Копийтер для MarkdownHeaderTextSplitter

### Создаем текстовый файл с диалогами из Гугл таблицы

In [7]:
from openpyxl import Workbook, load_workbook
from io import BytesIO
import urllib

# Шаг 1: Загрузить xlsx файл из Google Drive
google_sheet_url = FIRST_TEXTS[KEY]
file = urllib.request.urlopen(google_sheet_url).read()

wb=load_workbook(filename=BytesIO(file))
sheet_ranges=wb[FIRST_SHEETS[KEY]]

# Шаг 2: Извлечь столбец "text"
text_column = ""
for row in sheet_ranges.iter_rows():
    first = False
    last = False
    for cell in row:
        if not first and not last:
            text_column += str(cell.value) + "\n"
            last = True
        first = True

# Шаг 3: Заменить "operatorMessage: Здравствуйте" на "<operatorMessage: Здравствуйте>"
text_column = text_column.replace("operatorMessage: Здравствуйте", "<operatorMessage: Здравствуйте>")

# Шаг 4: Сохранить в файл "dialog.txt" в корне Colab
with open("/content/drive/MyDrive/KIA_DIALOG/Copywriter/KIA/BASE/kia_dialog_" + KEY + ".txt", "w", encoding="utf-8") as f:
        f.write(str(text_column) + "\n")

print("Файл kia_dialog_" + KEY + ".txt успешно сохранен!")

Файл kia_dialog_ONLY_6.txt успешно сохранен!


In [8]:
#@title Создаем объект для работы Copywriter
projects_dir = '/content/drive/MyDrive/KIA_DIALOG/Copywriter/KIA/'

Promt_copywriter = "https://docs.google.com/document/d/15bNgGsY2MPrgfnUasyHX0SslylqLGAm7/edit?usp=sharing"
Promt_lector = "https://docs.google.com/document/d/1N0PUuhI8zP8Iu_4kZIuF5pIht78hQvg2/edit?usp=sharing"

# если темы созданы
# topics_final = "/content/drive/MyDrive/Colab Notebooks/_Projects_ChatGPT/Нейро_Copywriter/УИИ/___.txt"
# # Создаем объект для дообучения chatGPT
curator = WorkerОpenAI(system_promt = Promt_copywriter, # системный промт
                       system_promt_lector = Promt_lector, # промт Лектора
                      #  content_topics = topics_final,
                       save_project = projects_dir)     # путь для сохранения готовых файлов

# Формируем базу по файлу txt
# путь к материалам
doc_txt_dir = projects_dir + 'BASE/'
db_initial = projects_dir

curator.f00_create_embedding_faiss_db(doc_txt_dir = doc_txt_dir,   # путь к материалам
                                faiss_db_dir = db_initial,     # путь для сохранения исходной базы
                                start_idx = 0,                 # номер документа в базе
                                collection_name = 'KIA_db_initial')  # наименование коллекции


Загружается файл:  kia_dialog_ONLY_6.txt
Количество токенов в документе : 916

ЦЕНА запроса создания базы индексов: 0.0003664  $


In [10]:
# curator.create_embedding_faiss_db(doc_txt_dir = doc_txt_dir,   # путь к материалам
#                                 faiss_db_dir = db_initial,     # путь для сохранения исходной базы
#                                 start_idx = 0,                 # номер документа в базе
#                                 collection_name = 'УИИ_db_initial')  # наименование коллекции


# curator.create_embedding_one_file(doc_txt_dir = doc_txt_dir,   # путь к материалам
#                                   file_ = file_name,            # какой файл берем
#                                   faiss_db_dir = projects_dir)     # путь для сохранения исходной базы

AttributeError: ignored

In [11]:
# Смотрим созданные чанки
print(f"Текст разбит на чанки. Всего: {len(curator.db.docstore._dict.values())} шт.\n")
curator.db.docstore._dict.values()

Текст разбит на чанки. Всего: 2 шт.



dict_values([Document(page_content='3.0\n7.0\n10.0\n11.0\n12.0\n17.0\n28.0\n29.0\n34.0\n35.0\n39.0\n41.0\n48.0\n51.0\n55.0\n62.0\n63.0\n67.0\n71.0\n77.0\n78.0\n81.0\n82.0\n90.0\n91.0\n96.0\n98.0\n99.0\n100.0\n106.0\n110.0\n111.0\n112.0\n113.0\n124.0\n126.0\n134.0\n143.0\n149.0\n151.0\n153.0\n154.0\n158.0\n165.0\n168.0\n174.0\n177.0\n179.0\n180.0\n182.0\n196.0\n198.0\n199.0\n201.0\n203.0\n213.0\n214.0\n221.0\n222.0\n223.0\n229.0\n234.0\n235.0\n240.0\n241.0\n242.0\n245.0\n249.0\n252.0\n255.0\n258.0\n262.0\n266.0\n272.0\n278.0\n282.0\n284.0\n285.0\n287.0\n292.0\n297.0\n304.0\n305.0\n313.0\n315.0\n318.0\n321.0\n322.0\n326.0\n327.0\n331.0\n335.0\n337.0\n341.0\n347.0\n349.0\n359.0\n360.0\n362.0\n363.0\nNone\nNone\nNone\nNone\nNone\nNone\nNone\nNone\nNone\nNone\nNone\nNone\nNone\nNone\nNone\nNone\nNone\nNone\nNone\nNone\nNone\nNone\nNone\nNone\nNone\nNone\nNone\nNone\nNone\nNone\nNone\nNone\nNone\nNone\nNone\nNone\nNone\nNone\nNone\nNone\nNone\nNone\nNone\nNone\nNone\nNone\nNone\nNone\nNone\n

In [12]:
#@title Поиск темы и подтемы выборкой по 5 - 15 чанков

# Подаем в цикле и просим составить список тем и подтем.
curator.search_topics_subtopics()

1195 токенов использовано на вопрос 

1830 токенов использовано всего (вопрос-ответ).
===========================================: 

1195 токенов использовано на вопрос 

1830 токенов использовано всего (вопрос-ответ).
===========================================: 

Собрали список тем и подтем: 
# Темы:

1. Отрывок материала из диалога с другими клиентами

## Подтемы:

- 3.0
- 7.0
- 10.0
- 11.0
- 12.0
- 17.0
- 28.0
- 29.0
- 34.0
- 35.0
- 39.0
- 41.0
- 48.0
- 51.0
- 55.0
- 62.0
- 63.0
- 67.0
- 71.0
- 77.0
- 78.0
- 81.0
- 82.0
- 90.0
- 91.0
- 96.0
- 98.0
- 99.0
- 100.0
- 106.0
- 110.0
- 111.0
- 112.0
- 113.0
- 124.0
- 126.0
- 134.0
- 143.0
- 149.0
- 151.0
- 153.0
- 154.0
- 158.0
- 165.0
- 168.0
- 174.0
- 177.0
- 179.0
- 180.0
- 182.0
- 196.0
- 198.0
- 199.0
- 201.0
- 203.0
- 213.0
- 214.0
- 221.0
- 222.0
- 223.0
- 229.0
- 234.0
- 235.0
- 240.0
- 241.0
- 242.0
- 245.0
- 249.0
- 252.0
- 255.0
- 258.0
- 262.0
- 266.0
- 272.0
- 278.0
- 282.0
- 284.0
- 285.0
- 287.0
- 292.0
- 297.0
- 304.0
- 3

In [13]:
#@title Корректируем список тем и подтем (обобщаем, убираем дубли).
# Подаем сформированный Список тем и просим объединить похожие по смыслу темы или подтемы, записать компактно.
# При необходимости перефразировать тему или подтему.
curator.get_merge_topics()

1579 токенов использовано на вопрос 

2115 токенов использовано всего (вопрос-ответ).
===========================================: 

Итоговый список тем и подтем: 
# Темы:

1. Отрывок материала из диалога с другими клиентами

## Подтемы:

- 3.0, 7.0, 10.0, 11.0, 12.0, 17.0, 28.0, 29.0, 34.0, 35.0, 39.0, 41.0, 48.0, 51.0, 55.0, 62.0, 63.0, 67.0, 71.0, 77.0, 78.0, 81.0, 82.0, 90.0, 91.0, 96.0, 98.0, 99.0, 100.0, 106.0, 110.0, 111.0, 112.0, 113.0, 124.0, 126.0, 134.0, 143.0, 149.0, 151.0, 153.0, 154.0, 158.0, 165.0, 168.0, 174.0, 177.0, 179.0, 180.0, 182.0, 196.0, 198.0, 199.0, 201.0, 203.0, 213.0, 214.0, 221.0, 222.0, 223.0, 229.0, 234.0, 235.0, 240.0, 241.0, 242.0, 245.0, 249.0, 252.0, 255.0, 258.0, 262.0, 266.0, 272.0, 278.0, 282.0, 284.0, 285.0, 287.0, 292.0, 297.0, 304.0, 305.0, 313.0, 315.0, 318.0, 321.0, 322.0, 326.0, 327.0, 331.0, 335.0, 337.0, 341.0, 347.0, 349.0, 359.0, 360.0, 362.0, 363.0


## ОРГАНИЗУЕМ ИТОГОВЫЙ ТЕКСТ

Выборка блоков документов по схожести с подтемой. Передаем собранные отрывки с Лекции и просим рассказать подробнее, опираясь на Лекцию.

In [ ]:
# запускаем сбору Итогового Текста
# name_file = 'Разбор нейро-сотрудников на chatGPT _ 21.08.2023'
# db_initial = '/content/drive/MyDrive/Colab Notebooks/_Projects_ChatGPT/Нейро_Copywriter/УИИ/УИИ_db_initial'

# curator.organize_final_text(name = name_file, db_path = db_initial)
curator.organize_final_text()

0it [00:00, ?it/s]


 ===========================================: 
1155 токенов использовано на вопрос 

2433 токенов использовано всего (вопрос-ответ).
===========================================: 


 ===========================================: 
1150 токенов использовано на вопрос 

2379 токенов использовано всего (вопрос-ответ).
===========================================: 


 ===========================================: 
1175 токенов использовано на вопрос 

2494 токенов использовано всего (вопрос-ответ).
===========================================: 


 ===========================================: 
1150 токенов использовано на вопрос 

2284 токенов использовано всего (вопрос-ответ).
===========================================: 


 ===========================================: 
1157 токенов использовано на вопрос 

2146 токенов использовано всего (вопрос-ответ).
===========================================: 


 ===========================================: 
1150 токенов использовано на вопрос 

2339 токе

In [ ]:
# сформирован итоговый текст
curator.final_text

# MarkdownHeaderTextSplitter

In [ ]:
#@title функции
chat_manager_system = """
Ты Специалист, готовящий текст для обучающего материала. Ты профессионал своего дела.
Тебе предоставят отрывок Твоего Текста и попросят раскрыть подробнее одну из тем обучающего материала.
Твоя цель: Опираясь только на Текст, подробно, развернуто рассказать по интересующей теме на русском языке.
Необходимо уложиться в 2000 токенов.
"""

def get_chatgpt_answer(topic,  db, model = MODEL_TURBO_0613):
  # Выборка документов по схожести с вопросом
  docs = db.similarity_search(topic, k=4)
  message_content = re.sub(r'\n{2}', ' ', '\n '.join([f'\n  ' + doc.page_content + '\n' for i, doc in enumerate(docs)]))
  # print('message_content :\n ======================================== \n', self.message_content)

  messages = [
    {"role": "system", "content": f"{chat_manager_system}"},
    {"role": "user", "content": f"""Analyze step by step and give a detailed correct answer to the Student's question.\n
    Question:\n{topic}\n\nMaterials from the Webinar:\n{message_content}\n\nAnswer:"""}
    ]

  try:
    completion = openai.ChatCompletion.create(
    model=model,
    messages=messages,
    temperature=0.1
    )

    print(f'{completion["usage"]["total_tokens"]} токенов использовано всего (вопрос-ответ).')
    print('ЦЕНА запроса с ответом :', 0.0015*(completion["usage"]["total_tokens"]/1000), ' $')
    print('===========================================: \n')
    print('Ответ ChatGPT: ')
    print(completion.choices[0].message.content)
    # return completion.choices[0].message.content
  except:
    print("Модель в настоящее время перегружена. Попробуйте позже.")

def load_txt_file(file_path):
    with open(file_path, 'r') as file_:
        text = file_.read()
    return text

projects_dir = '/content/drive/MyDrive/KIA_DIALOG/Copywriter/KIA/'
name_file = "kia_dialog_" + KEY + '.tx__final_text.txt' # FIXME
final_text_dir = projects_dir + name_file

# Загружаем итоговый текст
final_text = load_txt_file(final_text_dir)


In [ ]:
#@title  Готовим документ MarkdownHeader по сформированному тексту
from langchain.text_splitter import MarkdownHeaderTextSplitter

headers_to_split_on = [
    ("_#", "Header 1"),
    ("##_", "Header 2"),
    ("###_", "Header 3"),
    ("####_", "Header 4"),
    ("#####_", "Header 5"),
]

markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
md_header_splits = markdown_splitter.split_text(final_text)
md_header_splits

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1024, chunk_overlap=300
)

# Split
split = text_splitter.split_documents(md_header_splits)
db = FAISS.from_documents(split, OpenAIEmbeddings())

In [ ]:
split[1]

In [ ]:
topic1 = """
Почем сегодня тормозные колодки?
"""

get_chatgpt_answer(topic1,  db)

In [ ]:
topic2 = """
Почему сайт kia.ru не работает?
"""

get_chatgpt_answer(topic2,  db)

In [ ]:
topic3 = """
Какую длину и ширину имеет тормозная колодка?
"""

get_chatgpt_answer(topic3,  db)